In [ ]:
% matplotlib inline

from __future__ import division
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import time as tm
import gsw as sw
from scipy.optimize import curve_fit
from scipy.stats import chi2

import numpy.ma as ma

from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset as NetCDFFile

from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Times New Roman'})
#rc('text', usetex=True)

In [ ]:
def checklon(data):
    cxx = (data['lon']).values # longitude

    # convert all longitude to longitude east
    cxx[cxx<0]= 360+cxx[cxx<0];
    data['lon_e'] = cxx
    cxbew = cxx
    cxbew[cxbew>180] = cxbew[cxbew>180]-360   
    data['lon'] = cxbew
    return data

def distance(data):
    cruise = pd.unique(data['cruise'])
    distances = pd.DataFrame()    
    for c in cruise:
        temp = pd.DataFrame()
        temp = data.loc[(data['cruise']==c), ['cruise','file_time','lon','lat']]
        
        # calculate distance in km between points
        temp['dx'] = np.insert(sw.distance(temp['lon'].values, temp['lat'].values, 0),0,0)/1000
        temp['x'] = np.cumsum(temp['dx'])
        
        if c == cruise[0]:
            distances = distances.append(temp)
        else:
            distances = pd.concat([distances, temp], axis=0)
        del temp
    data = pd.merge(left = data, right = distances, how='left', left_on=['cruise','file_time','lat','lon'], right_on=['cruise','file_time','lat','lon'])
    #return distances

    return data  

def get_month(row):
    return tm.strptime(row['file_time'], '%Y-%m-%d %H:%M:%S')[1] 
    

In [ ]:
# import data

indir = '../data/'


In [ ]:
j = 0

thing = binned[:,:,j]
Zm = ma.masked_invalid(thing)

fig1 = plt.figure(3, figsize=(18,9), dpi = 600)
palette = plt.get_cmap('RdYlBu_r', 50)
m = Basemap(projection='merc',llcrnrlat=-50,urcrnrlat=60,\
            llcrnrlon=100,urcrnrlon=340,lat_ts=0,resolution='c')

m.drawcoastlines()
m.drawmapboundary(fill_color='white')
m.fillcontinents(color='grey',lake_color='white')
#plt.axes([0.025, 0.025, 0.95, 0.95])


xm, ym = m(xgrid, ygrid)
#cax = m.scatter(xm,ym,s=10, c=all_fronts['level'].values, cmap = palette, lw = 0, vmin = -0.5, vmax =4.5 , alpha = 0.5)
m.pcolormesh(xm, ym, (Zm), vmin = 0 ,vmax = 200, cmap = palette)

m.drawparallels(np.arange(-50,70,20),labels=[1,1,0,1], fontsize=24)
m.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1], fontsize=24)

plt.tick_params(axis='both', which='major', labelsize=24)
plt.tick_params(axis='both', which='minor', labelsize=24)
plt.xlabel('Longitude', fontsize = 24, labelpad = 40)
plt.ylabel('Latitude', fontsize = 24, labelpad = 70)

# cb = fig1.colorbar(cax)
# cb.ax.tick_params(labelsize=16)
#cb.set_label('Seaflow chlorophyll', fontsize=16)
cbar = plt.colorbar(pad = 0.08)
cbar.ax.tick_params(labelsize=24) 
#cbar.set_label('Prochlorococcus (10$^6$ cells L$^{-1}$)', fontsize=24, rotation = 90, labelpad = 20)
plt.show()

#fig1.savefig('../figures/seaflow_shannon_map_4deg.png', bbox_inches='tight', dpi = 600)